# Fine tuning averaging weights

Fine tuning models could dramatically enhance model's performance.

In the last notebook we have been trying to get the best of the blending technique by changing weights manually.
During this notebook we will write an optimization function.

In [1]:
import glob
import pandas as pd
import numpy as np

In [2]:
INPUT_DIR="../data-nlp/input/"
OUTPUT_DIR="../data-nlp/output/"

RAND=10

In [3]:
files = glob.glob(OUTPUT_DIR+"02_basemodel*.csv")

df = None

for f in files:
    if df is None:
        df = pd.read_csv(f)
    else:
        temp_df = pd.read_csv(f)
        df = df.merge(temp_df, on="id", how="left")

df.head(10)

,id,sentiment_x,kfold_x,lr_cnt_pred,Unnamed: 0,sentiment_y,kfold_y,lr_pred,sentiment,kfold,rf_svd_pred
0,11703_9,1,0,9.996652e-01,0,1,0,0.868421,1,0,0.57
1,6742_8,1,0,9.866726e-01,1,1,0,0.727763,1,0,0.59
2,10071_1,0,0,4.653006e-01,2,0,0,0.441153,0,0,0.54
3,9841_7,1,0,9.999992e-01,3,1,0,0.943808,1,0,0.73
4,11579_10,1,0,9.851556e-01,4,1,0,0.900753,1,0,0.51
5,5216_8,1,0,9.852354e-01,5,1,0,0.729515,1,0,0.51
6,1683_7,1,0,6.576467e-03,6,1,0,0.309059,1,0,0.40
7,818_4,0,0,9.940245e-01,7,0,0,0.713800,0,0,0.73
8,11779_3,0,0,2.116804e-02,8,0,0,0.307074,0,0,0.32
9,27_1,0,0,2.024757e-11,9,0,0,0.001033,0,0,0.12


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b1ab3b60-9130-40c7-8fa2-288950ad463c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>